In [ ]:
pip install lightning neurokit2 captum numpy pandas wfdb -q

In [ ]:
pip install wfdb

In [ ]:
import os, sys
import warnings; warnings.filterwarnings("ignore")
import pytorch_lightning; pytorch_lightning.seed_everything(22)

from tqdm import tqdm

import argparse
import random
import pandas as pd, numpy as np
import neurokit2 as nk
import torch
import torch.nn as nn, torch.optim as optim
import torch.nn.functional as F
import torch.nn.utils.prune as prune
import captum.attr as attr
import matplotlib.pyplot as pyplot
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score
import pywt
import h5py
import wfdb

In [ ]:
def thresholds_search(labels, preds):
    search_range = [round(t, 2) for t in np.arange(1, 20)*0.05]

    optimal_thresholds = []
    for cls in range(preds.shape[1]):
        f1_scores_cls = []
        for threshold in search_range:
            labels_cls, preds_cls = labels[:, cls], preds[:, cls]

            preds_cls = list(np.where(preds[:, cls] >= threshold, 1, 0))
            f1_scores_cls.append(f1_score(
                labels_cls, preds_cls
                , average = "macro"
            ))

        optimal_thresholds.append(search_range[np.argmax(f1_scores_cls)])

    return optimal_thresholds

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import zipfile
import re

# Define the path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/CINC2020.zip'

# Define the extraction path
extract_path = '/content/CINC'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Check if the zip file exists
if os.path.exists(zip_file_path):
    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Successfully extracted {zip_file_path} to {extract_path}")
else:
    print(f"Error: {zip_file_path} not found.")

In [ ]:
from concurrent.futures import ProcessPoolExecutor

# ---------------------------
# Denoising function (vectorized)
# ---------------------------
def denoise_ecg(signal, wavelet='db5', level=9):
    """
    Denoise ECG signal (multi-lead) using wavelet transform.
    signal: shape (n_leads, n_samples)
    """
    coeffs = pywt.wavedec(signal, wavelet, level=level, axis=1)
    # Universal threshold
    threshold = (np.median(np.abs(coeffs[-1])) / 0.6745) * (np.sqrt(2 * np.log(signal.shape[1])))

    # Zero out first two detail levels
    coeffs[-1].fill(0)
    coeffs[-2].fill(0)

    # Threshold other details
    for i in range(1, len(coeffs) - 2):
        coeffs[i] = pywt.threshold(coeffs[i], threshold)

    # Reconstruct
    rdata = pywt.waverec(coeffs, wavelet)

    return np.nan_to_num(rdata, nan=0.0, posinf=0.0, neginf=0.0)

# ---------------------------
# Pad/truncate helper
# ---------------------------
def pad_or_truncate(arr, target_len):
    """
    Pad or truncate signal to fixed length.
    arr: shape (n_leads, n_samples)
    """
    if arr.shape[1] < target_len:
        pad_width = target_len - arr.shape[1]
        return np.pad(arr, ((0, 0), (0, pad_width)), mode='constant')
    return arr[:, :target_len]


def process_record(file_name, folder, available_labels):
    final_data_path = f"CINC/classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/{folder}/{file_name}"
    hdr = wfdb.rdheader(final_data_path)

    for comment in hdr.comments:
        if comment.startswith('Dx:') and "," not in comment:
            label = comment.replace('Dx: ', '')
            if label not in available_labels:
                return None

            record = wfdb.rdrecord(final_data_path)
            signal = np.nan_to_num(record.p_signal, nan=0.0)

            signal = signal.T  # shape (n_leads, n_samples)

            denoised = denoise_ecg(signal)  # vectorized
            return denoised, label
    return None

def load_and_preprocess_data():
    print("Loading and preprocessing ECG data...")

    RECORDS = "CINC/classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/RECORDS"
    TARGET_LENGTH = 5000
    available_labels = [
        '10370003', '164889003', '164909002', '164934002',
        '270492004', '284470004', '426177001', '426783006',
        '427084000', '427393009', '59118001'
    ]

    X, Y = [], []

    with open(RECORDS) as f:
        folders_list = f.read()
        FOLDERS = folders_list.strip().splitlines()

    tasks = []
    with ProcessPoolExecutor() as ex:
        for folder in FOLDERS:
            if folder in ["training/ptb/g1/", "training/st_petersburg_incart/g1/"]:
                continue

            with open(f"CINC/classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/{folder}/RECORDS") as r:
                files = r.read().strip().splitlines()[:-1]

            for file_name in files:
                tasks.append(ex.submit(process_record, file_name, folder, available_labels))

        results = [t.result() for t in tasks if t.result() is not None]

    for signal, label in results:
        truncated = pad_or_truncate(signal, TARGET_LENGTH)
        X.append(truncated)
        Y.append(label)

    print("Data loading finished")

    # Convert to arrays
    X = np.array(X)
    Y = np.array(Y)

    # One-hot encoding (faster than pandas)
    label2idx = {label: i for i, label in enumerate(available_labels)}
    Y_idx = np.array([label2idx[y] for y in Y])
    Y_one_hot = np.eye(len(available_labels))[Y_idx]

    return X, Y_one_hot

In [ ]:
standardized_arrays, Y_one_hot_final = load_and_preprocess_data()

# standardized_arrays = standardized_arrays.transpose(0, 2, 1)

In [ ]:
standardized_arrays.shape, Y_one_hot_final.shape

In [ ]:
X = torch.tensor(standardized_arrays, dtype=torch.float32)   # (14530, 5000, 1)
y = torch.tensor(np.array(Y_one_hot_final).argmax(axis=1), dtype=torch.long)  # (14530,)

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# ---- Stratified splitting ----
all_indices = list(range(len(X)))
all_labels = y  # label_idx for each sample

# First split: train+val (90%) vs test (10%)
trainval_indices, test_indices = train_test_split(
    all_indices,
    test_size=0.1,
    random_state=10,
    stratify=all_labels
)

trainval_labels = [all_labels[i] for i in trainval_indices]

# Second split: train (90% of trainval) vs val (10% of trainval)
train_indices, val_indices = train_test_split(
    trainval_indices,
    test_size=0.1,
    random_state=10,
    stratify=trainval_labels
)

print(np.array(X).shape, np.array(y).shape)

# Create datasets
train_ds = TensorDataset(torch.Tensor(X[train_indices]), torch.Tensor(y[train_indices]))
val_ds = TensorDataset(torch.Tensor(X[val_indices]), torch.Tensor(y[val_indices]))
test_ds = TensorDataset(torch.Tensor(X[test_indices]), torch.Tensor(y[test_indices]))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)


In [ ]:

class LightSEModule(nn.Module):
    def __init__(self,
        in_channels,
        reduction = 16,
    ):
        super(LightSEModule, self).__init__()
        self.pool = nn.AdaptiveAvgPool1d(1)

        self.s_conv = DSConv1d(
            in_channels, in_channels//reduction,
            kernel_size = 1,
        )
        self.act_fn = nn.ReLU()
        self.e_conv = DSConv1d(
            in_channels//reduction, in_channels,
            kernel_size = 1,
        )

    def forward(self,
        input,
    ):
        attention_scores = self.pool(input)

        attention_scores = self.s_conv(attention_scores)
        attention_scores = self.act_fn(attention_scores)
        attention_scores = self.e_conv(attention_scores)

        return input*torch.sigmoid(attention_scores)

class DSConv1d(nn.Module):
    def __init__(self,
        in_channels, out_channels,
        kernel_size, padding = 0, stride = 1,
    ):
        super(DSConv1d, self).__init__()
        self.dw_conv = nn.Conv1d(
            in_channels, in_channels,
            kernel_size = kernel_size, padding = padding, stride = stride,
            groups = in_channels,
            bias = False,
        )
        self.pw_conv = nn.Conv1d(
            in_channels, out_channels,
            kernel_size = 1,
            bias = False,
        )

    def forward(self,
        input,
    ):
        output = self.dw_conv(input)
        output = self.pw_conv(output)

        return output

class LightSEResBlock(nn.Module):
    def __init__(self,
        in_channels,
        downsample = False,
    ):
        super(LightSEResBlock, self).__init__()
        if downsample:
            self.out_channels = in_channels*2
            self.conv_1 = DSConv1d(
                in_channels, self.out_channels,
                kernel_size = 7, padding = 3, stride = 2,
            )
            self.identity = nn.Sequential(
                DSConv1d(
                    in_channels, self.out_channels,
                    kernel_size = 1, padding = 0, stride = 2,
                ),
                nn.BatchNorm1d(self.out_channels),
            )
        else:
            self.out_channels = in_channels
            self.conv_1 = DSConv1d(
                in_channels, self.out_channels,
                kernel_size = 7, padding = 3, stride = 1,
            )
            self.identity = nn.Identity()
        self.conv_2 = DSConv1d(
            self.out_channels, self.out_channels,
            kernel_size = 7, padding = 3, stride = 1,
        )

        self.convs = nn.Sequential(
            self.conv_1,
            nn.BatchNorm1d(self.out_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            self.conv_2,
            nn.BatchNorm1d(self.out_channels),
            LightSEModule(self.out_channels),
        )
        self.act_fn = nn.ReLU()

    def forward(self,
        input,
    ):
        output = self.convs(input) + self.identity(input)
        output = self.act_fn(output)

        return output

class LightSEResNet18(nn.Module):
    def __init__(self,
        base_channels = 64,
    ):
        super(LightSEResNet18, self).__init__()
        self.bblock = LightSEResBlock
        self.stem = nn.Sequential(
            nn.Conv1d(
                1, base_channels,
                kernel_size = 15, padding = 7, stride = 2,
            ),
            nn.BatchNorm1d(base_channels),
            nn.ReLU(),
            nn.MaxPool1d(
                kernel_size = 3, padding = 1, stride = 2,
            ),
        )
        self.stage_0 = nn.Sequential(
            self.bblock(base_channels),
            self.bblock(base_channels),
        )

        self.stage_1 = nn.Sequential(
            self.bblock(base_channels*1, downsample = True),
            self.bblock(base_channels*2),
        )
        self.stage_2 = nn.Sequential(
            self.bblock(base_channels*2, downsample = True),
            self.bblock(base_channels*4),
        )
        self.stage_3 = nn.Sequential(
            self.bblock(base_channels*4, downsample = True),
            self.bblock(base_channels*8),
        )

        self.pool = nn.AdaptiveAvgPool1d(1)

    def forward(self,
        input,
    ):
        output = self.stem(input)
        output = self.stage_0(output)

        output = self.stage_1(output)
        output = self.stage_2(output)
        output = self.stage_3(output)

        output = self.pool(output)

        return output

In [ ]:

class LightX3ECG(nn.Module):
    def __init__(self,
        base_channels = 64,
        num_classes = 11,
    ):
        super(LightX3ECG, self).__init__()
        self.backbone_0 = LightSEResNet18(base_channels)
        self.backbone_1 = LightSEResNet18(base_channels)
        self.backbone_2 = LightSEResNet18(base_channels)
        self.lw_attention = nn.Sequential(
            nn.Linear(
                base_channels*24, base_channels*8,
            ),
            nn.BatchNorm1d(base_channels*8),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(
                base_channels*8, 3,
            ),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(
                base_channels*8, num_classes,
            ),
        )

    def forward(self,
        input,
        return_attention_scores = False,
    ):
        features_0 = self.backbone_0(input[:, 0, :].unsqueeze(1)).squeeze(2)
        features_1 = self.backbone_1(input[:, 1, :].unsqueeze(1)).squeeze(2)
        features_2 = self.backbone_2(input[:, 2, :].unsqueeze(1)).squeeze(2)
        attention_scores = torch.sigmoid(
            self.lw_attention(
                torch.cat(
                [
                    features_0,
                    features_1,
                    features_2,
                ],
                dim = 1,
                )
            )
        )
        merged_features = torch.sum(
            torch.stack(
            [
                features_0,
                features_1,
                features_2,
            ],
            dim = 1,
            )*attention_scores.unsqueeze(-1),
            dim = 1,
        )

        output = self.classifier(merged_features)

        if not return_attention_scores:
            return output
        else:
            return output, attention_scores

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

def train_fn(
    train_loaders,
    model,
    num_epochs,
    config,
    criterion,
    optimizer,
    scheduler = None,
    training_verbose = True,
):
    print("\nStart Training ...\n" + " = "*16)
    model = model.cuda()
    model = nn.DataParallel(model, device_ids = config["device_ids"])

    best_f1 = 0.0
    for epoch in tqdm(range(1, num_epochs + 1), disable = training_verbose):
        if training_verbose:
          print("epoch {:2}/{:2}".format(epoch, num_epochs) + "\n" + "-"*16)

        model.train()
        running_loss = 0.0
        running_labels, running_preds = [], []

        for ecgs, labels in tqdm(train_loaders["train"], disable = not training_verbose):
            ecgs, labels = ecgs.cuda(), labels.cuda()

            optimizer.zero_grad()
            logits = model(ecgs)
            loss = criterion(logits, labels)

            loss.backward()
            optimizer.step()

            running_loss = running_loss + loss.item()*ecgs.size(0)
            labels, preds = list(labels.data.cpu().numpy()), list(torch.max(logits, 1)[1].detach().cpu().numpy()) if not config["is_multilabel"] else list(np.where(torch.sigmoid(logits).detach().cpu().numpy() >= 0.50, 1, 0))
            running_labels.extend(labels), running_preds.extend(preds)

        if (scheduler is not None) and (not epoch > scheduler.T_max):
            scheduler.step()

        epoch_loss, epoch_f1 = running_loss/len(train_loaders["train"].dataset), f1_score(
            running_labels, running_preds
            , average = "weighted"
        )

        if training_verbose:
            print("{:<5} - loss:{:.4f}, f1:{:.4f}".format(
                "train",
                epoch_loss, epoch_f1
            ))

        # Validation
        with torch.no_grad():
            model.eval()
            running_loss = 0.0
            running_labels, running_preds = [], []
            for ecgs, labels in tqdm(train_loaders["val"], disable = not training_verbose):
                ecgs, labels = ecgs.cuda(), labels.cuda()

                logits = model(ecgs)
                loss = criterion(logits, labels)

                running_loss = running_loss + loss.item()*ecgs.size(0)
                labels, preds = list(labels.data.cpu().numpy()), list(torch.max(logits, 1)[1].detach().cpu().numpy()) if not config["is_multilabel"] else list(np.where(torch.sigmoid(logits).detach().cpu().numpy() >= 0.50, 1, 0))
                running_labels.extend(labels), running_preds.extend(preds)

            val_epoch_loss, val_epoch_f1 = running_loss/len(train_loaders["val"].dataset), f1_score(
                running_labels, running_preds
                , average = "weighted"
            )

            val_acc = accuracy_score(running_labels, running_preds)
            val_precision = precision_score(running_labels, running_preds, average="weighted", zero_division=0)
            val_recall = recall_score(running_labels, running_preds, average="weighted", zero_division=0)
            val_f1 = f1_score(running_labels, running_preds, average="weighted", zero_division=0)



        if training_verbose:
            print("{:<5} - loss:{:.4f}, f1:{:.4f}, precision:{:.4f}, recall:{:.4f}, accuracy:{:.4f},".format(
                "val",
                val_epoch_loss, val_epoch_f1, val_precision, val_recall, val_acc
            ))


In [ ]:
def set_seed(seed):
    # Python's built-in random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

In [ ]:
test_f1s = []
test_accs = []
test_precisions = []
test_recalls = []
test_aucs = []

for seed in range(0, 10):
  config = {
      "is_multilabel":False,
      "device_ids": [0],
  }

  set_seed(seed)


  # ---- Stratified splitting ----
  all_indices = list(range(len(X)))
  all_labels = y  # label_idx for each sample

  # First split: train+val (90%) vs test (10%)
  trainval_indices, test_indices = train_test_split(
      all_indices,
      test_size=0.1,
      random_state=seed,
      stratify=all_labels
  )

  trainval_labels = [all_labels[i] for i in trainval_indices]

  # Second split: train (90% of trainval) vs val (10% of trainval)
  train_indices, val_indices = train_test_split(
      trainval_indices,
      test_size=0.1,
      random_state=seed,
      stratify=trainval_labels
  )

  # Create datasets
  train_ds = TensorDataset(torch.Tensor(X[train_indices]), torch.Tensor(y[train_indices]))
  val_ds = TensorDataset(torch.Tensor(X[val_indices]), torch.Tensor(y[val_indices]))
  test_ds = TensorDataset(torch.Tensor(X[test_indices]), torch.Tensor(y[test_indices]))

  train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)
  val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
  test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

  config = {
      "is_multilabel":False,
      "device_ids": [0],
  }

  train_loaders = {
      "train": train_loader,
      "val": val_loader,
      "test": test_loader
  }


  model = LightX3ECG(
      num_classes = 11,
  )

  # if not config["is_multilabel"]:
  criterion = F.cross_entropy
  # else:
  # criterion = F.binary_cross_entropy_with_logits

  optimizer = optim.Adam(
      model.parameters(),
      lr = 3e-5, weight_decay = 1e-5,
  )

  scheduler = optim.lr_scheduler.CosineAnnealingLR(
      optimizer,
      eta_min = 1e-6, T_max = 20,
  )

  train_fn(
      train_loaders,
      model,
      num_epochs = 15,
      config = config,
      criterion = criterion,
      optimizer = optimizer,
      scheduler = scheduler,
  )


  # Validation
  with torch.no_grad():
      model.eval()
      running_loss = 0.0
      running_labels, running_preds, running_probs = [], [], []
      for ecgs, labels in tqdm(test_loader):
          ecgs, labels = ecgs.cuda(), labels.cuda()

          logits = model(ecgs)
          loss = criterion(logits, labels)
          probs = torch.softmax(logits, dim=1).cpu().numpy()
          running_loss = running_loss + loss.item()*ecgs.size(0)
          labels, preds = list(labels.data.cpu().numpy()), list(torch.max(logits, 1)[1].detach().cpu().numpy()) if not config["is_multilabel"] else list(np.where(torch.sigmoid(logits).detach().cpu().numpy() >= 0.50, 1, 0))
          running_labels.extend(labels), running_preds.extend(preds), running_probs.extend(probs)

      test_epoch_loss, test_epoch_f1 = running_loss/len(test_loader.dataset), f1_score(
          running_labels, running_preds
          , average = "weighted"
      )

      test_acc = accuracy_score(running_labels, running_preds)
      test_precision = precision_score(running_labels, running_preds, average="weighted", zero_division=0)
      test_recall = recall_score(running_labels, running_preds, average="weighted", zero_division=0)
      test_f1 = f1_score(running_labels, running_preds, average="weighted", zero_division=0)
      # Some classes may be missing in a split, handle safely
      try:
        y_true = np.eye(11)[running_labels]
        y_score = np.array(running_probs)

        auc_list = []
        for i in range(11):
            if np.any(y_true[:, i]):  # class i exists
                auc_list.append(roc_auc_score(y_true[:, i], y_score[:, i]))
        if auc_list:
            auc = np.mean(auc_list)
        else:
            auc = float("nan")
      except ValueError:
          auc = float("nan")

      print("{:<5}, f1:{:.4f}, precision:{:.4f}, recall:{:.4f}, accuracy:{:.4f},".format(
                  "eval",
                  test_f1, test_precision, test_recall, test_acc
              ))

      test_accs.append(test_acc)
      test_precisions.append(test_precision)
      test_recalls.append(test_recall)
      test_f1s.append(test_f1)
      test_aucs.append(auc)

print(test_accs)
print(test_precisions)
print(test_recalls)
print(test_f1s)
print(test_aucs)

In [ ]:
test_f1s, test_accs,test_precisions,test_recalls,test_aucs


In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary

model = LightX3ECG(
    num_classes = 11,
)

summary(model, (1, 12, 5000))

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# ---- Stratified splitting ----
all_indices = list(range(len(X)))
all_labels = y  # label_idx for each sample

# First split: train+val (90%) vs test (10%)
trainval_indices, test_indices = train_test_split(
    all_indices,
    test_size=0.1,
    random_state=10,
    stratify=all_labels
)

trainval_labels = [all_labels[i] for i in trainval_indices]

# Second split: train (90% of trainval) vs val (10% of trainval)
train_indices, val_indices = train_test_split(
    trainval_indices,
    test_size=0.1,
    random_state=10,
    stratify=trainval_labels
)

print(np.array(X).shape, np.array(Y).shape)

# Create datasets
train_ds = TensorDataset(torch.Tensor(X[train_indices]), torch.Tensor(y[train_indices]))
val_ds = TensorDataset(torch.Tensor(X[val_indices]), torch.Tensor(y[val_indices]))
test_ds = TensorDataset(torch.Tensor(X[test_indices]), torch.Tensor(y[test_indices]))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

config = {

    "is_multilabel":False,
    "device_ids": [0],
}

train_loaders = {
    "train": train_loader,
    "val": val_loader,
    "test": test_loader
}



# Validation
with torch.no_grad():
    model.eval()
    running_loss = 0.0
    running_labels, running_preds = [], []
    for ecgs, labels in tqdm(test_loader):
        ecgs, labels = ecgs.cuda(), labels.cuda()

        logits = model(ecgs)
        loss = criterion(logits, labels)

        running_loss = running_loss + loss.item()*ecgs.size(0)
        labels, preds = list(labels.data.cpu().numpy()), list(torch.max(logits, 1)[1].detach().cpu().numpy()) if not config["is_multilabel"] else list(np.where(torch.sigmoid(logits).detach().cpu().numpy() >= 0.50, 1, 0))
        running_labels.extend(labels), running_preds.extend(preds)

    test_epoch_loss, test_epoch_f1 = running_loss/len(test_loader.dataset), f1_score(
        running_labels, running_preds
        , average = "weighted"
    )

    test_acc = accuracy_score(running_labels, running_preds)
    test_precision = precision_score(running_labels, running_preds, average="weighted", zero_division=0)
    test_recall = recall_score(running_labels, running_preds, average="weighted", zero_division=0)
    test_f1 = f1_score(running_labels, running_preds, average="weighted", zero_division=0)

    print("{:<5}, f1:{:.4f}, precision:{:.4f}, recall:{:.4f}, accuracy:{:.4f},".format(
                "eval",
                test_f1, test_precision, test_recall, test_acc
            ))


In [ ]:
pip install torchinfo

In [ ]:
!pip install calflops -q

In [ ]:
import calflops


flops, macs, params = calflops.calculate_flops(model=model,
                                      input_shape=(1, 12, 5000),
                                      output_as_string=True,
                                      output_precision=4)
print("Model FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))

In [ ]:
def analyze_model_complexity(model, input_size=(1, 12, 5000), device='cuda'):
    """
    Analyze model complexity: parameters, FLOPs, memory

    Args:
        model: Your HANWithAttention model
        input_size: (batch, segments, timesteps, channels)
        device: 'cuda' or 'cpu'
    """
    print("="*80)
    print("MODEL COMPLEXITY ANALYSIS")
    print("="*80)

    model = model.to(device)
    model.eval()

    # 1. Parameter count
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params

    print(f"\n📊 Parameter Count:")
    print(f"   Total Parameters:       {total_params:,}")
    print(f"   Trainable Parameters:   {trainable_params:,}")
    print(f"   Non-trainable Parameters: {non_trainable_params:,}")
    print(f"   Model Size (MB):        {total_params * 4 / (1024**2):.2f}")  # 4 bytes per float32

    # 2. FLOPs calculation
    dummy_input = torch.randn(input_size).to(device)

    try:
        macs, params = profile(model, inputs=(dummy_input,), verbose=False)
        macs, params = clever_format([macs, params], "%.3f")
        print(f"\n🔢 FLOPs (Floating Point Operations):")
        print(f"   MACs (Multiply-Accumulate): {macs}")
        print(f"   Parameters (thop):          {params}")
    except Exception as e:
        print(f"\n⚠️  FLOPs calculation failed: {e}")

    # 3. Layer-wise parameter breakdown
    print(f"\n📋 Layer-wise Parameter Breakdown:")
    print(f"{'Layer Name':<40} {'Parameters':>15} {'% of Total':>12}")
    print("-"*70)

    for name, param in model.named_parameters():
        if param.requires_grad:
            param_count = param.numel()
            percentage = 100 * param_count / trainable_params
            print(f"{name:<40} {param_count:>15,} {percentage:>11.2f}%")

    # 4. Detailed model summary
    print(f"\n📝 Detailed Model Architecture:")
    summary(model,
            input_size=input_size,
            col_names=["input_size", "output_size", "num_params", "mult_adds"],
            depth=4,
            device=device,
            verbose=0)

    return {
        'total_params': total_params,
        'trainable_params': trainable_params,
        'model_size_mb': total_params * 4 / (1024**2)
    }

analyze_model_complexity(model)

In [ ]:
import time
import numpy as np
import torch

# =============================================================================
# Benchmark Environment Setup
# =============================================================================

def setup_benchmark_env():
    """
    Freeze backend behavior for reproducible benchmarking.
    """
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False

def time_forward(model, inputs, device='cuda'):
    """
    Time a single forward pass with proper synchronization.
    Returns elapsed time in seconds.
    """
    if device == 'cuda':
        torch.cuda.synchronize()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        _ = model(inputs)
        end.record()

        torch.cuda.synchronize()
        return start.elapsed_time(end) / 1000.0  # seconds
    else:
        t0 = time.time()
        _ = model(inputs)
        return time.time() - t0


# =============================================================================
# Batch-1 Latency Benchmark
# =============================================================================

def benchmark_latency(
    model,
    input_shape,
    device='cuda',
    runs=100,
    warmup=20
):
    """
    Measure true per-sample inference latency (batch size = 1).
    """
    model = model.to(device).eval()

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(input_shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(input_shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "mean_ms": times.mean() * 1000,
        "std_ms": times.std() * 1000,
        "median_ms": np.median(times) * 1000,
        "min_ms": times.min() * 1000,
        "max_ms": times.max() * 1000,
    }


# =============================================================================
# Batch-N Throughput Benchmark
# =============================================================================

def benchmark_throughput(
    model,
    input_shape,
    batch_size,
    device='cuda',
    runs=100,
    warmup=20
):
    """
    Measure throughput under batched inference.
    """
    model = model.to(device).eval()
    shape = (batch_size,) + input_shape[1:]

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "batch_time_ms": times.mean() * 1000,
        "per_sample_ms": (times.mean() / batch_size) * 1000,
        "samples_per_sec": batch_size / times.mean()
    }

def run_inference_benchmark(
    model,
    input_shape=(1, 10, 300, 12),
    batch_size=16,
    device='cuda'
):
    setup_benchmark_env()

    print("=" * 80)
    print("INFERENCE BENCHMARK")
    print("=" * 80)
    print(f"Device: {device}")
    print(f"Input shape: {input_shape}")
    print(f"Throughput batch size: {batch_size}")

    latency = benchmark_latency(
        model=model,
        input_shape=input_shape,
        device=device
    )

    throughput = benchmark_throughput(
        model=model,
        input_shape=input_shape,
        batch_size=batch_size,
        device=device
    )

    print("\nBatch-1 Latency (ms):")
    for k, v in latency.items():
        print(f"  {k:>10}: {v:.3f}")

    print("\nBatch-N Throughput:")
    for k, v in throughput.items():
        if "ms" in k:
            print(f"  {k:>16}: {v:.3f}")
        else:
            print(f"  {k:>16}: {v:.2f}")

    return {
        "latency": latency,
        "throughput": throughput
    }

results = run_inference_benchmark(
    model,
    input_shape=(1, 12, 5000),
    batch_size=16,
    device='cuda'
)


In [ ]:
results = run_inference_benchmark(
    model,
    input_shape=(1, 12, 5000),
    batch_size=16,
    device='cpu'
)